### 서울시 공원 시각화

In [2]:
import numpy as np
import pandas as pd
import folium


In [3]:
park = pd.read_csv('data/서울시 주요 공원현황.csv', encoding='euc-kr')
print(park.shape)
park.info()

(132, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            132 non-null    int64  
 1   공원명           132 non-null    object 
 2   공원개요          131 non-null    object 
 3   면적            129 non-null    object 
 4   개원일           122 non-null    object 
 5   주요시설          124 non-null    object 
 6   주요식물          88 non-null     object 
 7   안내도           102 non-null    object 
 8   오시는길          115 non-null    object 
 9   이용시참고사항       120 non-null    object 
 10  이미지           132 non-null    object 
 11  지역            131 non-null    object 
 12  공원주소          132 non-null    object 
 13  관리부서          132 non-null    object 
 14  전화번호          132 non-null    object 
 15  X좌표(GRS80TM)  122 non-null    float64
 16  Y좌표(GRS80TM)  122 non-null    float64
 17  X좌표(WGS84)    131 non-null    float64
 18  Y좌표(WGS84)    131 no

In [4]:
park = park[['공원명', '면적', '공원주소', 'Y좌표(WGS84)', 'X좌표(WGS84)' ]]
park.columns = ['공원명', '면적', '공원주소', '위도', '경도']
park.head(1)

,공원명,면적,공원주소,위도,경도
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동),37.55014,126.990377


In [5]:
park.isna().sum()

공원명     0
면적      3
공원주소    0
위도      1
경도      1
dtype: int64

In [6]:
park.dropna(subset=['면적', '위도'], inplace=True)
park.isna().sum()

공원명     0
면적      0
공원주소    0
위도      0
경도      0
dtype: int64

In [7]:
park.면적.head(80).tail(10)

70       130345㎡
71      5332422㎡
72      5107964㎡
73        47832㎡
74       213552㎡
75    168099.87㎡
76     359435.2㎡
77        36660㎡
78      380735㎡ 
79    1807420.9㎡
Name: 면적, dtype: object

In [8]:
park.면적[6]

'휴양 및 편익시설 : 평의자 7개 화장실 23.18㎡ 조경시설 - 은하폭포(높이 12m × 폭10m 발원지로부터 17.3m 폭 1.5 ~ 10m 수조 255㎡ 담수량 140t) - 산성폭포(높이 5m×폭 12m 수조 90㎡ 담수량 75t)- 명속의 벽 1개소(높이 5.4m) 교양시설 : 야외공연무대 102㎡ 관리시설 : 관리실 8.55㎡ 전기실 51.03㎡ 기계실 83.36㎡ 안내판 1개 공원 등 18본 휴양시설 : 벤취 12개 '

- 면적에 따라 공원크기를 지도에 표시

In [9]:
park.drop([6], inplace=True)

In [10]:
# 총 144m 같은 글에서 '총  ' 지우기
park.면적 = park.면적.str.replace('총 ', '')

In [11]:
# 앞에 있는 숫자만 추출하기 - 구분자, ㎡, m , ㎥
# 구분자가 여러개인 경우 split()
import re 
re.split('[,.]', 'a,b.c')

['a', 'b', 'c']

In [12]:
park.면적 = park.면적.apply(lambda x: re.split('[㎡m㎥]', x)[0])
park.면적.head()

0    2896887
1      80683
2    9132690
3     480994
4    2284085
Name: 면적, dtype: object

In [13]:
park.면적 = park.면적.astype(float).astype(int)

In [14]:
park

,공원명,면적,공원주소,위도,경도
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),37.550140,126.990377
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),37.540394,127.154779
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,37.426449,127.019846
3,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),37.543072,127.041798
4,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),37.571805,126.878907
...,...,...,...,...,...
126,서일대뒷산공원,631021,서울특별시 중랑구 면목동 산19,37.587716,127.098782
127,용마도시자연공원(사가정공원),109635,서울특별시 중랑구 면목7동 산50,37.579762,127.095997
128,문화비축기지,140022,서울특별시 마포구 증산로 87,37.571718,126.893246
129,경춘선숲길,211392,노원구 공릉2동 산 82-2,37.627077,127.076482


- 공원 크기에 따라 분류

In [15]:
# 기준: 소형 < 100_000, 중형 < 1_000_000, 대형
criteria = [0, 100000, 1000000, 200000000]
labels = ['소형', '중형', '대형']   # 분류 기준
size_info = [3, 7, 15]              # CircleMarker의 크기

park['분류'] = pd.cut(park.면적, criteria, labels=labels) 
park['크기'] = pd.cut(park.면적, criteria, labels=size_info).astype(int)
park.head()


,공원명,면적,공원주소,위도,경도,분류,크기
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),37.550140,126.990377,대형,15
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),37.540394,127.154779,소형,3
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,37.426449,127.019846,대형,15
3,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),37.543072,127.041798,중형,7
4,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),37.571805,126.878907,대형,15


In [16]:
park.to_csv('data/서울공원요약.csv', index=False)

- 공원 시각화

In [17]:
parkmap = folium.Map(location=[37.55, 126.98], zoom_start=12)

for i in park.index:
    folium.CircleMarker(
        location=[park.위도[i], park.경도[i]],
        radius=int(park['크기'][i]),
        # radius=300,
        tooltip=park.공원명[i],
        popup=folium.Popup(park.공원주소[i], max_width=200),
        color='crimson', 
        fill=True
    ).add_to(parkmap)
title_html='<h3 align="center" style="font-size:20px;">서울시 공원</h3>'
parkmap.get_root().html.add_child(folium.Element(title_html))
parkmap
